In [1]:
from numpy import sin, cos, arccos, arctan2, sqrt, pi, zeros, ones, asarray, radians, arange, fabs
import numpy
npmax = numpy.max
from numpy.linalg import norm
from urllib.request import urlopen
from shutil import copyfileobj
from collections import namedtuple

from gnss.time import gpsseconds, gpstime, utctime
from gnss.coordinates import ecef2sky, geo2ecef
from gnss.orbits import compute_gps_satellite_position_from_yuma
from gnss.visualization import SkyPlot

there are 27 leap second epochs


In [2]:
from bokeh.plotting import output_server, show, hplot, Session

In [3]:
session = Session(root_url='http://192.168.3.137:5006/', load_from_config=False)
# session.register('anon', '1234')
session.login('anon', '1234')
output_server('almanac orbits', session=session)

In [4]:
sky_plot = SkyPlot()
show(sky_plot.plot)
# call `sky_plot.update(...)` to update plot

In [5]:
year = 2015
almanac_filename = 'almanac.yuma.week0810.319488.txt'
almanac_filepath = '../../data/' + almanac_filename
almanac_url = 'http://celestrak.com/GPS/almanac/Yuma/{0}/{1}'.format(year, almanac_filename) 
with urlopen(almanac_url) as in_stream, open(almanac_filepath, 'wb') as out_file:
    copyfileobj(in_stream, out_file)

In [6]:
print(almanac_url)

http://celestrak.com/GPS/almanac/Yuma/2015/almanac.yuma.week0810.319488.txt


In [7]:
# !cat data/almanac.yuma.week0810.319488.txt

In [8]:
# parse almanac
YumaAlmanac = namedtuple('YumaAlmanac', 'health e t_oa i_0 Omega_dot sqrt_a Omega_week omega M_0 A_f0 A_f1 week')
almanacs = {}
with open(almanac_filepath, 'r') as f:
    lines = f.readlines()
    index = 0
    while index < len(lines):
        if lines[index].startswith('ID'):
            svid = int(lines[index][25:])
            health = int(lines[index + 1][25:])
            e = float(lines[index + 2][25:])
            t_oa = float(lines[index + 3][25:])
            i_0 = float(lines[index + 4][25:])
            Omega_dot = float(lines[index + 5][25:])
            sqrt_a = float(lines[index + 6][25:])
            Omega_week = float(lines[index + 7][25:])
            omega = float(lines[index + 8][25:])
            M_0 = float(lines[index + 9][25:])
            A_f0 = float(lines[index + 10][25:])
            A_f1 = float(lines[index + 11][25:])
            week = int(lines[index + 12][25:])
            almanacs[svid] = YumaAlmanac(health, e, t_oa, i_0, Omega_dot, sqrt_a, Omega_week, omega, M_0, A_f0, A_f1, week)
            index += 13
        index += 1
print(almanacs[2])

YumaAlmanac(health=0, e=0.01445627213, t_oa=319488.0, i_0=0.940823973, Omega_dot=-7.977475151e-09, sqrt_a=5153.72998, Omega_week=-0.4596073839, omega=-2.307922781, M_0=1.544693936, A_f0=0.0005512237549, A_f1=3.637978807e-12, week=810)


In [9]:
# ephemeric/almanac parameters
svid = 1
alm = almanacs[svid]
# GPS week
week = alm.week
# eccentricity
e = alm.e
# time of applicability
t_oa = alm.t_oa
# orbital inclination at reference time
i_0 = alm.i_0
# rate of right ascension
Omega_dot = alm.Omega_dot
# semi-major axis??
a = alm.sqrt_a**2
# right ascension at start of week
Omega_week = alm.Omega_week
# argument of perigee
omega = alm.omega
# mean anomaly at reference time
M_0 = alm.M_0
# `A_f0` and `A_f1` are clock correction parameters
# get time of applicability/ephemeris
print('gps week and week seconds: {0}   {1:7.3f}'.format(week, t_oa))
week, t_oe = gpstime(utctime(t_oa, week + 1024)).week_and_week_seconds
print('actual week and week seconds: {0}   {1:7.3f}'.format(week, t_oe))

gps week and week seconds: 810   319488.000
actual week and week seconds: 1834   319488.000


In [10]:
# ephemeris-only parameters
# harmonic correction parameters (not available in almanac)
c_us = c_rs = c_is = c_uc = c_rc = c_ic = 0.
# mean anomaly offset
delta_n = 0.
# rate of inclination angle
i_dot = 0.

In [11]:
# generate user times
t = gpsseconds(week, t_oe) + arange(0., 6. * 3600., 60.)
# get time delta from ephemeris epoch
delta_t = t - gpsseconds(week, t_oe)

In [12]:
# Earth gravitational constant (m^3/s^2)
mu = 3.986005e14
# WGS Earth rotation rate (rad/s)
Omega_e_dot = 7.2921151467e-5
# mean motion (rad/s)
n_0 = sqrt(mu / a**3)
# corrected mean motion
n = n_0 + delta_n
# mean anomaly
M = M_0 + n * delta_t
# eccentric anomaly
# we minimize (E + e * sin(E) - M)**2; derivative with respect to E is 2 * (E + e * sin(E) - M) * (1 + e * cos(E))
E = 0.
err = 1.
tol = 1e-10
while err > tol:
    _temp = M + e * sin(E)
    err = npmax(fabs(_temp - E))
    E = _temp
# true anomaly
nu = arctan2(sqrt(1. - e**2) * sin(E), cos(E) - e)
# eccentric anomaly
E = arccos((e + cos(nu)) / (1. + e * cos(nu)))
# argument of latitude
Phi = nu + omega
# secord-order harmonic perturbation corrections
delta_u = c_us * sin(2 * Phi) + c_uc * cos(2 * Phi)
delta_r = c_rs * sin(2 * Phi) + c_rc * cos(2 * Phi)
delta_i = c_is * sin(2 * Phi) + c_ic * cos(2 * Phi)
# argument of latitude (corrected)
u = Phi + delta_u
# orbital radius (corrected)
r = a * (1. - e * cos(E)) + delta_r
# inclination (corrected)
i = i_0 + delta_i + i_dot * delta_t
# longitude of ascensing node (corrected)
Omega = Omega_week + (Omega_dot - Omega_e_dot) * delta_t - Omega_e_dot * t_oe

In [13]:
# compute x,y in orbital plane
x_orb = r * cos(u)
y_orb = r * sin(u)
# compute x,y velocity in orbital plane
v_x_orb = n * a * sin(E) / (1. - e * cos(E))
v_y_orb = -n * a * sqrt(1. - e**2) * cos(E) / (1. - e * cos(E))
# transform from orbital system to ECEF system
x_ecef = x_orb * cos(Omega) - y_orb * sin(Omega) * cos(i)
y_ecef = x_orb * sin(Omega) + y_orb * cos(Omega) * cos(i)
z_ecef = y_orb * sin(i)
# put into array
sv_ecef = asarray([x_ecef, y_ecef, z_ecef])

In [14]:
print('radius of satellite orbit: {0}'.format(norm(sv_ecef, axis=0)[10]))
print(t[10], sv_ecef[:, 10])

radius of satellite orbit: 26510892.47501812
1109523288.0 [-14306555.54144303   5367015.46551261  21664372.44312714]


-1.9258e7, -7.6316e6, 1.6673e7

325504

Hong Kong coordinates:

$$22^o 12’ 34.2647’’, 114^o 15’ 28.5115’’$$

In [15]:
rx_geo = [45.209517972, 114.257919861, 0.0]
# rx_geo = [22.209517972, 114.257919861, 0.0]
rx_ecef = geo2ecef(rx_geo)

In [52]:
svid = 19
almanac = almanacs[svid]
t = gpsseconds(almanac.week, almanac.t_oa, rollover=1) + arange(0., 6 * 3600, 60)
sv_ecef = compute_gps_satellite_position_from_yuma(almanac, t, rollover=1)
sky = ecef2sky(rx_ecef, sv_ecef)
print('radius of satellite orbit: {0}'.format(norm(sv_ecef, axis=0)[10]))
print(t[10], sv_ecef[:, 10])

radius of satellite orbit: 26786143.414283298
1109523288.0 [-25235573.55869067  -8254911.87577486   3538323.9107883 ]


In [53]:
sky_plot.update(svid, sky[::10, 0], sky[::10, 1])

(684,) (684,) 269
(269,) (269,)


In [18]:
az = sky[::6, 0]
el = sky[::6, 1]
r = 90. - el
theta = radians(az)
ind = r > 90
r = r[ind]
theta = theta[ind]
sky_plot.update(svid, r, theta)

In [65]:
visibility = np.zeros((32, len(t),))

.

.

.

.

Below, we consolidate this functionality into one function.

In [3]:
# // implemented under gnss/orbits/gps_orbits.py

In [68]:
from matplotlib.pyplot import rc, grid, figure, plot, rcParams, savefig
from numpy import radians

class SkyPlot:
    
    def __init__(self, fig=None, where=(0.1, .1, .8, .8)):
        if not fig:
            # force square figure and square axes looks better for polar, IMO
            width, height = rcParams['figure.figsize']
            size = min(width, height)
            # make a square figure
            fig = figure(figsize=(size, size))
        rc('grid', color='#316931', linewidth=1, linestyle='-')
        rc('xtick', labelsize=15)
        rc('ytick', labelsize=15)
        self.fig = fig
        self.ax = self.fig.add_axes(where, polar=True, axisbg='#d5de9c')
        self.ax.set_theta_zero_location('N')
        self.ax.set_theta_direction(-1)
        self.ax.set_yticks(range(0, 90+10, 10))                   # Define the yticks
        yLabel = ['90', '', '', '60', '', '', '30', '', '', '']
        self.ax.set_yticklabels(yLabel)
        self.ax.grid(True)
        
    def plot(self, azimuth, elevation, prn=' '):
        prn = str(prn)
        self.ax.annotate(prn,
                        xy=(np.deg2rad(azimuth[0]), 90 - elevation[0]),  # theta, radius
                        bbox=dict(boxstyle='round', fc=(.9,.95,.99), alpha = 0.8),
                        horizontalalignment='center',
                        verticalalignment='center')
        self.ax.scatter(np.deg2rad(azimuth), 90 - elevation)
        self.ax.set_ylim(0, 100)


In [85]:
import matplotlib.pyplot as plt

fig = plt.figure()

skyplot = SkyPlot(fig, where=(.1, .1, .4, .7))

for i, svid in enumerate(range(1, 33)):
    if svid not in almanacs.keys():
        continue
    alm = almanacs[svid]
    sv_ecef = compute_gps_satellite_position_from_yuma(alm, t)
    sky = coordinates.ecef2sky(rx_ecef, sv_ecef)
    skyplot.plot(sky[:, 0], sky[:, 1], prn=svid)
    visibility[i, :] = sky[:, 1] > 0.  # elevation is greater than zero

ax = fig.add_axes([.6, .1, .35, .35])
n, bins = np.histogram(np.sum(visibility, axis=0), bins=np.arange(0, 20, 1))
n = n / np.sum(n)
ax.bar(bins[:-1], n)
ax.set_title('probability of # visible satellites')
ax.set_xlabel('# satellites')
ax.set_ylabel('probability')

ax = fig.add_axes([.6, .55, .35, .35])
for svid in range(1, 33):
    time = t.copy() - t[0]
    time[visibility[svid-1, :] == 0] = np.nan
    ax.scatter(time, svid * np.ones((len(time),)))
ax.set_title('visibility')
ax.set_xlabel('time (s)')
ax.set_ylabel('prn')

plt.show()
plt.close()

In [81]:
svid = 1
alm = almanacs[svid]
sv_ecef = compute_gps_satellite_position_from_yuma(alm, t)
sky = coordinates.ecef2sky(rx_ecef, sv_ecef)
print(sv_ecef[:, 0])
print(sky[0, :])
print(t[0])
print(week)

0.995656399738
[-13982294.22872636   6984513.59443379  21403749.70731708]
[ 32.92919031  35.43179518]
319488.0
1834
